# Model inference

1. Download model and batch inference data
2. Make predictions
3. Store predictions in a monitoring feature group

### Connect to Hopsworks

In [177]:
import hopsworks
import xgboost as xgb
import unicodedata
import re
from xgboost import XGBRegressor

# connect with Hopsworks
project = hopsworks.login(
        host="eu-west.cloud.hopsworks.ai",
        project="ID2223_Project",
        api_key_value=os.environ["HOPSWORKS_API_KEY"]
    )

# Get feature view
fs = project.get_feature_store()
fv = fs.get_feature_view('avalanche_warning_fv_new_corrected_more_features_and_lags', version=5)

# Get model registry
mr = project.get_model_registry()

2026-01-10 19:24:05,526 INFO: Closing external client and cleaning up certificates.
2026-01-10 19:24:05,530 INFO: Connection closed.
2026-01-10 19:24:05,532 INFO: Initializing external client
2026-01-10 19:24:05,532 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-10 19:24:07,033 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2173


In [178]:
def sanitize_name(name):
    # Normalize Unicode to ASCII, ignore accents
    name_ascii = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode()
    # Replace anything not a-z, A-Z, 0-9, or _ with underscore
    name_clean = re.sub(r'[^a-zA-Z0-9_]', '_', name_ascii)
    return name_clean

def predict(model: xgb.XGBRegressor, features_df: pd.DataFrame) -> float:
    """
    Predict avalanche risk
    """
    features_df = features_df.astype(float)
    return float(model.predict(features_df)[0])

In [179]:
# Create batch data for the feature view
batch_data = fv.get_batch_data(dataframe_type="pandas")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.28s) 


### Download models from Model Registry

In [34]:
from xgboost import XGBRegressor

#Retrieve the name resorts
resorts = {loc: None for loc in batch_data["location"].unique()}

models = {}           
model_dirs = {}      

for loc in resorts.keys():  
    loc_ = sanitize_name(loc.replace(" ", "_"))
    print(f"Loading model for {loc}...")

    # Retrieve model from registry
    model = mr.get_model(
        name=f"xgb_avalanche_model_{loc_}",
        version=3  
    )

    print(model)
    
    # Download model artifacts
    model_dir = model.download()

    # Load XGBoost model
    xgb_model = XGBRegressor()
    xgb_model.load_model(
        f"{model_dir}/xgb_ordinal_model_more_features{loc_}.json"
    )

    # Store everything
    models[loc] = xgb_model
    model_dirs[loc] = model_dir

    print(f"✓ Model for {loc} loaded successfully\n")


Loading model for Bjorli Ski...
Model(name: 'xgb_avalanche_model_Bjorli_Ski', version: 3)


Downloading: 0.000%|          | 0/399508 elapsed<00:00 remaining<?

✓ Model for Bjorli Ski loaded successfullys)... DONE

Loading model for Eikedalen Ski Center AS...
Model(name: 'xgb_avalanche_model_Eikedalen_Ski_Center_AS', version: 3)


Downloading: 0.000%|          | 0/366544 elapsed<00:00 remaining<?

✓ Model for Eikedalen Ski Center AS loaded successfully

Loading model for Galdhøpiggen Summer Ski Centre...
Model(name: 'xgb_avalanche_model_Galdhpiggen_Summer_Ski_Centre', version: 3)


...

✓ Model for Sauda Ski Centre loaded successfullyDONE

Loading model for Strandafjellet Skisenter...
Model(name: 'xgb_avalanche_model_Strandafjellet_Skisenter', version: 3)


Downloading: 0.000%|          | 0/366828 elapsed<00:00 remaining<?

✓ Model for Strandafjellet Skisenter loaded successfully

Loading model for Voss Resort Fjellheisar...
Model(name: 'xgb_avalanche_model_Voss_Resort_Fjellheisar', version: 3)


Downloading: 0.000%|          | 0/601204 elapsed<00:00 remaining<?

✓ Model for Voss Resort Fjellheisar loaded successfully



### Get Weather Forecast Features with Feature View

In [187]:
# Feature group for weather
aq_fg = fs.get_feature_group(
    name='weather_terrain_sensor',
    version=2,
)

aq_df = aq_fg.read().sort_values(by="date", ascending=False)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.89s) 


In [199]:
latest_7_per_location_weather

,date,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,location,snow_load_steep,wind_snow_transport,rain_on_snow_risk,temp_elev,precip_slope_weighted
22170,2026-01-10 00:00:00+00:00,-19.944500,0.0,0.0,0.00,3.240000,270.000000,Hemsedal Skisenter,0.000000,0.382837,0.0,-11.462846,0.000000
22175,2026-01-10 00:00:00+00:00,-13.586500,0.0,0.0,0.00,1.800000,216.869980,Galdhøpiggen Summer Ski Centre,0.000000,0.346514,0.0,-7.039772,0.000000
22174,2026-01-10 00:00:00+00:00,-16.488001,0.0,0.0,0.00,1.938659,21.801476,Voss Resort Fjellheisar,0.000000,0.292608,0.0,-4.482924,0.000000
22173,2026-01-10 00:00:00+00:00,-11.367001,0.0,0.0,0.00,2.160000,180.000000,Eikedalen Ski Center AS,0.000000,0.349119,0.0,-4.378534,0.000000
22164,2026-01-10 00:00:00+00:00,-19.271000,0.0,0.0,0.00,0.804985,333.435028,Hovden Alpinsenter,0.000000,0.060642,0.0,-9.584816,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22238,2026-01-16 00:00:00+00:00,-6.094500,0.1,0.0,0.07,9.746631,94.236320,Hemsedal Skisenter,0.004434,1.151657,0.0,-3.502736,1.639558
22237,2026-01-16 00:00:00+00:00,-0.936500,0.0,0.0,0.00,3.319036,167.471207,Galdhøpiggen Summer Ski Centre,0.000000,0.638939,0.0,-0.485242,0.000000
22236,2026-01-16 00:00:00+00:00,-6.522000,0.0,0.0,0.00,9.290511,125.537766,Bjorli Ski,0.000000,1.379674,0.0,-4.316296,0.000000
22240,2026-01-16 00:00:00+00:00,-0.252500,0.0,0.0,0.00,13.397612,149.300354,Strandafjellet Skisenter,0.000000,2.870462,0.0,-0.037345,0.000000


In [220]:
# Sort batch data by date
batch_data_sorted = batch_data.sort_values(
    by="date", 
    ascending=False
)

# DataFrame of resorts considering only the most recent data
latest_7_per_location_weather = (
    aq_df
    .sort_values("date", ascending=False)
    .groupby("location", as_index=False)
    .head(7)
)

# Get todays value first
latest_7_per_location_weather = (latest_7_per_location_weather.sort_values(by="date",ascending=True))   

# DataFrame for the latest warnings for each resort
a = len(resorts)
df_warning_lag = batch_data_sorted.head(a)

# Dictionary to hold a DataFrame for each location
dfs_per_location = {}

# Loop over unique locations
for i, location in enumerate(latest_7_per_location_weather['location'].unique()):
    loc_ = sanitize_name(location)
    # Create a copy for the dictionary
    dfs_per_location[location] = latest_7_per_location_weather[latest_7_per_location_weather['location'] == location].copy()
    
    # Dynamically create a variable for each dataframe
    globals()[f'df_location_{loc_}'] = dfs_per_location[location]

# Feature columns
feature_cols = [
    "warning_level_lag_1", 
    "warning_level_lag_2",
    "warning_level_lag_3",
    "temperature_2m_mean",
    "precipitation_sum",
    "rain_sum",
    "snowfall_sum",
    "wind_speed_10m_max",
    "wind_direction_10m_dominant",
    "snow_load_steep",
    "wind_snow_transport",
    "rain_on_snow_risk",
    "temp_elev",
    "precip_slope_weighted",
]

In [221]:
df_warning_lag

,location,date,warning_level_lag_1,warning_level_lag_2,warning_level_lag_3,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,snow_load_steep,wind_snow_transport,rain_on_snow_risk,temp_elev,precip_slope_weighted
21958,Narvik Ski Resort,2026-01-10 00:00:00+00:00,3,3,2,-11.965500,0.0,0.0,0.0,12.722830,64.885201,0.0,0.381053,0.0,-0.198933,0.0
17818,Myrkdalen Fjellandsby,2026-01-10 00:00:00+00:00,2,2,2,-14.146000,0.0,0.0,0.0,0.804985,116.564987,0.0,0.100291,0.0,-7.597323,0.0
21957,Hovden Alpinsenter,2026-01-10 00:00:00+00:00,2,1,1,-19.271000,0.0,0.0,0.0,0.804985,333.435028,0.0,0.060642,0.0,-9.584816,0.0
8272,Sauda Ski Centre,2026-01-10 00:00:00+00:00,2,2,2,-17.589001,0.0,0.0,0.0,6.379216,73.610382,0.0,1.104451,0.0,-6.093398,0.0
20585,Hemsedal Skisenter,2026-01-10 00:00:00+00:00,1,1,1,-19.944500,0.0,0.0,0.0,3.240000,270.000000,0.0,0.382837,0.0,-11.462846,0.0
2767,Nedre fjellheisstasjon Narvik,2026-01-10 00:00:00+00:00,3,3,2,-11.965500,0.0,0.0,0.0,12.722830,64.885201,0.0,0.345439,0.0,-0.272582,0.0
11008,Galdhøpiggen Summer Ski Centre,2026-01-10 00:00:00+00:00,2,1,1,-13.586500,0.0,0.0,0.0,1.800000,216.869980,0.0,0.346514,0.0,-7.039772,0.0
5540,Voss Resort Fjellheisar,2026-01-10 00:00:00+00:00,2,2,2,-16.488001,0.0,0.0,0.0,1.938659,21.801476,0.0,0.292608,0.0,-4.482924,0.0
5539,Strandafjellet Skisenter,2026-01-10 00:00:00+00:00,2,2,2,-9.702499,0.0,0.0,0.0,5.400000,143.130020,0.0,1.156959,0.0,-1.434999,0.0
12370,Rauland Skisenter,2026-01-10 00:00:00+00:00,2,1,1,-14.759500,0.0,0.0,0.0,2.305125,321.340179,0.0,0.002630,0.0,-6.963444,0.0


In [223]:
# Add the lag features for today per location
for location in resorts.keys():
    loc_ = sanitize_name(location.replace(" ", "_"))
    df_name = f'df_location_{loc_}'
    df = globals()[df_name]

    #Reset index
    df.reset_index(drop=True, inplace=True)

    df['warning_level_lag_1'] = np.nan
    df['warning_level_lag_2'] = np.nan
    df['warning_level_lag_3'] = np.nan

    df_warning_lag_idx = df_warning_lag.set_index('location')

    if location in df_warning_lag_idx.index:
        df.loc[df.index[0],
               ['warning_level_lag_1',
                'warning_level_lag_2',
                'warning_level_lag_3']] = (
            df_warning_lag_idx.loc[location,
                ['warning_level_lag_1',
                 'warning_level_lag_2',
                 'warning_level_lag_3']]
        )

In [226]:
df_location_Bjorli_Ski

,date,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,location,snow_load_steep,wind_snow_transport,rain_on_snow_risk,temp_elev,precip_slope_weighted,warning_level_lag_1,warning_level_lag_2,warning_level_lag_3
0,2026-01-10 00:00:00+00:00,-18.421999,0.0,0.0,0.00,1.800000,126.869987,Bjorli Ski,0.00000,0.267306,0.0,-12.191783,0.000000,2.0,2.0,2.0
1,2026-01-11 00:00:00+00:00,-16.071999,0.0,0.0,0.00,4.896530,126.027466,Bjorli Ski,0.00000,0.727152,0.0,-10.636540,0.000000,NaN,NaN,NaN
2,2026-01-12 00:00:00+00:00,-13.172000,0.0,0.0,0.00,14.689588,126.027466,Bjorli Ski,0.00000,2.181456,0.0,-8.717304,0.000000,NaN,NaN,NaN
3,2026-01-13 00:00:00+00:00,-12.172000,0.0,0.0,0.00,9.449572,130.364456,Bjorli Ski,0.00000,1.403295,0.0,-8.055498,0.000000,NaN,NaN,NaN
4,2026-01-14 00:00:00+00:00,-9.472000,0.2,0.0,0.14,12.904882,120.141304,Bjorli Ski,0.01134,1.916421,0.0,-6.268623,3.442139,NaN,NaN,NaN
5,2026-01-15 00:00:00+00:00,-9.522000,0.0,0.0,0.00,5.351785,137.726379,Bjorli Ski,0.00000,0.794759,0.0,-6.301714,0.000000,NaN,NaN,NaN
6,2026-01-16 00:00:00+00:00,-6.522000,0.0,0.0,0.00,9.290511,125.537766,Bjorli Ski,0.00000,1.379674,0.0,-4.316296,0.000000,NaN,NaN,NaN


### Making the predictions

In [231]:
for location in resorts.keys():
    loc_ = sanitize_name(location.replace(" ", "_"))
    df = globals()[f'df_location_{loc_}']

    # Initialize lags from first row
    lag_1 = df.loc[0, 'warning_level_lag_1']
    lag_2 = df.loc[0, 'warning_level_lag_2']
    lag_3 = df.loc[0, 'warning_level_lag_3']

    for i, idx in enumerate(df.index[:7]):

        # Assign current lags to this row
        df.loc[idx, 'warning_level_lag_1'] = lag_1
        df.loc[idx, 'warning_level_lag_2'] = lag_2
        df.loc[idx, 'warning_level_lag_3'] = lag_3

        # Build features
        features = df.loc[[idx], feature_cols]

        # Predict
        prediction = predict(models[location], features)

        # Shift lags for next day
        lag_3, lag_2, lag_1 = lag_2, lag_1, prediction
        df['days_before_forecast_day'] = range(1, len(df) + 1)

### Store prediction values into feature stores

In [ ]:
monitor_fg_Bjorli_Ski = fs.get_or_create_feature_group(
    name='aq_predictions_Bjorli_Ski',
    description='Avalanche prediction monitoring with lags',
    version=1,
    primary_key=['location','date','days_before_forecast_day'],
    event_time="date"
)

monitor_fg_Bjorli_Ski.insert(df_location_Bjorli_Ski, wait=True)

In [235]:
import re

def sanitize_fg_name(name):
    # lowercase, replace spaces and non-alphanum with _
    name = name.lower()
    name = re.sub(r'[^a-z0-9]', '_', name)
    name = re.sub(r'_+', '_', name)  # collapse multiple underscores
    name = name.strip('_')  # remove leading/trailing underscores
    return name[:63]  # truncate to 63 chars


In [236]:
for location in resorts.keys():
    loc_ = sanitize_name(location.replace(" ", "_"))
    fg_name = sanitize_fg_name(f'aq_predictions_{loc_}')

    monitor_fg = fs.get_or_create_feature_group(
        name=fg_name,
        description='Avalanche prediction monitoring with lags',
        version=1,
        primary_key=['location', 'date', 'days_before_forecast_day'],
        event_time='date'
    )

    df = globals()[f'df_location_{loc_}']
    monitor_fg.insert(df, wait=True)


Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3387
2026-01-10 20:03:16,106 INFO: Computing insert statistics
Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3388
2026-01-10 20:03:30,623 INFO: Computing insert statistics
Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3389
2026-01-10 20:03:45,202 INFO: Computing insert statistics
Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3390
2026-01-10 20:04:01,140 INFO: Computing insert statistics
Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3391
2026-01-10 20:04:17,381 INFO: Computing insert statistics
Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3392
2026-01-10 20:04:32,093 INFO: Co